# Assignment 3: GPGPU Monte Carlo
## CSC 490: Lock-free, GPU & vectorization
### Victor Kamel

## Environment Setup

(Tested in Google Colab.)

> ⚠️ Must select a GPU runtime to run this code. `Runtime > Change runtime type > Hardware accelerator: "T4 GPU"`

In [1]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-9qpyk03k
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-9qpyk03k
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0a71d56e5dce3ff1f0dd2c47c29367629262f527
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4293 sha256=0f8a64a849abb13add77c404e03162f65e34a7c03d3a1ba2787f7ac0d4c95898
  Stored in directory: /tmp/pip-ephem-wheel-cache-dquq184j/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin


In [2]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


## Source Code

In [3]:
%%cuda --name monte_carlo_sim.cu

#define _USE_MATH_DEFINES

#include <stdlib.h>
#include <iostream>
#include <utility>
#include <chrono>
#include <random>
#include <cuda.h>
#include <thread>
#include <vector>
#include <math.h>

#define CUDA_CALL(exp)                                       \
    do {                                                     \
        cudaError res = (exp);                               \
        if(res != cudaSuccess) {                             \
            printf("Error at %s:%d\n %s\n",                  \
                __FILE__,__LINE__, cudaGetErrorString(res)); \
           exit(EXIT_FAILURE);                               \
        }                                                    \
    } while(0)

#define CHECK_ERROR(msg)                                             \
    do {                                                             \
        cudaError_t err = cudaGetLastError();                        \
        if(cudaSuccess != err) {                                     \
            printf("Error (%s) at %s:%d\n %s\n",                     \
                (msg), __FILE__, __LINE__, cudaGetErrorString(err)); \
            exit(EXIT_FAILURE);                                      \
        }                                                            \
    } while (0)

// Tausworth Parameters
constexpr unsigned TW_NTHREADS_PER_BLOCK = 1024;
constexpr unsigned TW_NBLOCKS            = 64;

constexpr unsigned TW_SAMPLES_PER_THREAD = 4096;
constexpr unsigned TW_NSEEDS_PER_RNG     = 4;

constexpr unsigned TW_NTHREADS = TW_NTHREADS_PER_BLOCK * TW_NBLOCKS;
constexpr unsigned TW_NSEEDS   = TW_NTHREADS * TW_NSEEDS_PER_RNG;
constexpr unsigned TW_SAMPLES  = TW_SAMPLES_PER_THREAD * TW_NTHREADS;

__device__ unsigned d_result;

// From GPU Gems 3 Ch. 37 (Lee Howes and David B. Thomas)
__device__ unsigned TausStep(unsigned &z, int S1, int S2, int S3, unsigned M)
{
    unsigned b = (((z << S1) ^ z) >> S2);
    return z = (((z & M) << S3) ^ b);
}

// From GPU Gems 3 Ch. 37 (Lee Howes and David B. Thomas)
__device__ unsigned LCGStep(unsigned &z)
{
    return z = (1664525 * z + 1013904223);
}

// From GPU Gems 3 Ch. 37 (Lee Howes and David B. Thomas)
__device__ float getRandomValueTauswortheUniform(unsigned &z1, unsigned &z2,
                                                 unsigned &z3, unsigned &z4)
{
    unsigned taus = TausStep(z1, 13, 19, 12, 4294967294UL)
                    ^ TausStep(z2, 2, 25, 4, 4294967288UL)
                    ^ TausStep(z3, 3, 11, 17, 4294967280UL);
    unsigned lcg = LCGStep(z4);

    return 2.3283064365387e-10f * (taus ^ lcg);	// taus + lcg
}

// Block-wise shared memory parallel reduce based on (Meister et al. 2023)
template <class T>
__device__ void reduceInto(T* loc, T val)
{
    // Prepare shared memory
    __shared__ unsigned smem[TW_NTHREADS_PER_BLOCK];
    smem[threadIdx.x] = val;

    __syncthreads(); // Synchronize

    for (int i = 1; i < blockDim.x; i <<= 1)
    {
        if (threadIdx.x < (threadIdx.x ^ i))
            { smem[threadIdx.x] += smem[threadIdx.x ^ i]; }
        __syncthreads(); // Synchronize
    }

    // Coalesce
	if (threadIdx.x == 0) { atomicAdd(loc, smem[0]); }
}

__global__ void runSimulation(unsigned *seedPool)
{
    unsigned z1, z2, z3, z4;
    unsigned addr = (blockIdx.x * blockDim.x + threadIdx.x) * TW_NSEEDS_PER_RNG;
    z1 = seedPool[addr    ]; z2 = seedPool[addr + 1];
    z3 = seedPool[addr + 2]; z4 = seedPool[addr + 3];

    // Note: We will not save RNG state for future invocations

    float x, y;
    unsigned count = 0;
    for (unsigned loop = 0; loop < TW_SAMPLES_PER_THREAD; ++loop)
    {
        x = getRandomValueTauswortheUniform(z1, z2, z3, z4);
        y = getRandomValueTauswortheUniform(z1, z2, z3, z4);
        count += (x * x) + (y * y) <= 1.0f;
    }

    reduceInto(&d_result, count);
}

std::pair<unsigned, unsigned> calculate_samples_gpu()
{
    // Initialize seed pool (seeds must be at least 128)
    unsigned seedPool[TW_NSEEDS];

    std::mt19937 gen(std::random_device{}());

    for (std::size_t i = 0; i < TW_NSEEDS; ++ i)
        do { seedPool[i] = gen(); } while (seedPool[i] < 128);

    // CUDA
    unsigned *devPool;

    CUDA_CALL(cudaMalloc((void **) &devPool, sizeof(unsigned) * TW_NSEEDS));
    CUDA_CALL(cudaMemcpy(devPool, seedPool, sizeof(unsigned) * TW_NSEEDS, cudaMemcpyHostToDevice));

    // Run simulation
    dim3 tw_grid(TW_NBLOCKS, 1, 1);
    dim3 tw_threads(TW_NTHREADS_PER_BLOCK, 1, 1);
    runSimulation <<< tw_grid, tw_threads >>> (devPool);

    CHECK_ERROR("Kernel execution failed.");

    typeof(d_result) count;
    cudaMemcpyFromSymbol(&count, d_result, sizeof(count), 0, cudaMemcpyDeviceToHost);

    CUDA_CALL(cudaFree(devPool));

    return std::pair<unsigned, unsigned>(count, TW_SAMPLES);
}

constexpr unsigned MT_TARGET_SAMPLES = 268435456;

std::pair<unsigned, unsigned> calculate_samples_cpu()
{
    const std::size_t n_threads = std::thread::hardware_concurrency();
    const std::size_t samples_per_thread = ceil(MT_TARGET_SAMPLES / n_threads);
    const std::size_t n_samples = samples_per_thread * n_threads;

    std::vector<std::size_t> samples(n_threads);

    {
        // Thread "pool"
        std::vector<std::jthread> spool;
        spool.reserve(n_threads);

        // Start threads
        for (std::size_t n = 0; n < n_threads; ++n) {
            spool.emplace_back([&samples, samples_per_thread, n] {
                std::random_device rd;
                std::mt19937 gen(rd());
                std::uniform_real_distribution<float> dist(-1.0f, 1.0f);

                std::size_t count = 0;
                for (std::size_t i = 0; i < samples_per_thread; ++i) {
                    float x = dist(gen);
                    float y = dist(gen);
                    count += (x * x) + (y * y) <= 1.0f;
                }
                samples[n] = count;
            });
        }
    }

    return std::pair<unsigned, unsigned>(
        std::accumulate(samples.cbegin(), samples.cend(), 0),
        n_samples
    );
}

int main()
{
    static_assert(sizeof(unsigned) == 4);

#ifdef GPU
    std::cout << "Threads / Block: " << TW_NTHREADS_PER_BLOCK << std::endl;
    std::cout << "Blocks: " << TW_NBLOCKS << std::endl;
#elif CPU
    std::cout << "Threads: " << std::thread::hardware_concurrency() << std::endl;
#else
#error("Provide a platform [GPU, CPU].")
#endif

    std::cout << std::endl;

    auto begin = std::chrono::high_resolution_clock::now();
#ifdef GPU
    auto [hit, total] = calculate_samples_gpu();
#elif CPU
    auto [hit, total] = calculate_samples_cpu();
#endif
    auto end = std::chrono::high_resolution_clock::now();
    auto duration = std::chrono::duration_cast<std::chrono::milliseconds>(end - begin).count();

    float PI = static_cast<float>(4 * hit) / static_cast<float>(total);

    std::cout << "Time (ms): " << duration << std::endl;
    std::cout << "Samples: " << total << std::endl;
    std::cout << "PI: " << PI << std::endl;
    std::cout << "MSE: " << (PI - M_PI) * (PI - M_PI) << std::endl;

    return EXIT_SUCCESS;
}


'File written in /content/src/monte_carlo_sim.cu'

### Compile and Run (GPU)

In [4]:
!nvcc -o /content/src/gpu /content/src/monte_carlo_sim.cu -DGPU -std=c++20

In [10]:
!/content/src/gpu

Threads / Block: 1024
Blocks: 64

Time (ms): 126
Samples: 268435456
PI: 3.14157
MSE: 4.36535e-10


### Compile and Run (CPU)

In [11]:
!nvcc -o /content/src/cpu /content/src/monte_carlo_sim.cu -DCPU -std=c++20

In [16]:
!/content/src/cpu

Threads: 2

Time (ms): 19686
Samples: 268435456
PI: 3.1414
MSE: 3.66197e-08


### Profiling

In [5]:
!nvprof /content/src/gpu

Threads / Block: 1024
Blocks: 64

==603== NVPROF is profiling process 603, command: /content/src/gpu
Time (ms): 425
Samples: 268435456
PI: 3.14145
MSE: 2.13344e-08
==603== Profiling application: /content/src/gpu
==603== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   98.51%  5.9493ms         1  5.9493ms  5.9493ms  5.9493ms  runSimulation(unsigned int*)
                    1.45%  87.646us         1  87.646us  87.646us  87.646us  [CUDA memcpy HtoD]
                    0.04%  2.1760us         1  2.1760us  2.1760us  2.1760us  [CUDA memcpy DtoH]
      API calls:   50.59%  132.47ms         1  132.47ms  132.47ms  132.47ms  cudaMalloc
                   46.91%  122.83ms         1  122.83ms  122.83ms  122.83ms  cudaLaunchKernel
                    2.28%  5.9589ms         1  5.9589ms  5.9589ms  5.9589ms  cudaMemcpyFromSymbol
                    0.10%  267.66us         1  267.66us  267.66us  267.66us  cudaMemcpy
              

In [ ]:
!ncu /content/src/gpu

Threads / Block: 1024
Blocks: 64

==PROF== Connected to process 3301 (/content/src/gpu)
==PROF== Profiling "runSimulation(unsigned int *)" - 0: 0%....50%....100% - 8 passes
Time (ms): 959
Samples: 268435456
PI: 3.14161
MSE: 2.35506e-10
==PROF== Disconnected from process 3301
[3301] gpu@127.0.0.1
  runSimulation(unsigned int *) (64, 1, 1)x(1024, 1, 1), Context 1, Stream 7, Device 0, CC 7.5
    Section: GPU Speed Of Light Throughput
    ----------------------- ------------- ------------
    Metric Name               Metric Unit Metric Value
    ----------------------- ------------- ------------
    DRAM Frequency          cycle/nsecond         5.00
    SM Frequency            cycle/usecond       585.19
    Elapsed Cycles                  cycle    3,476,367
    Memory Throughput                   %         0.11
    DRAM Throughput                     %         0.07
    Duration                      msecond         5.94
    L1/TEX Cache Throughput             %         0.14
    L2 Cache Th

In [ ]:
!nvidia-smi

Fri Dec 22 02:30:20 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--